In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [1]:
import numpy as np

import ccm_coda, kernel
import kernel
import tensorflow as tf

import pickle, os

from tools import * # we use cv_score, zero_replacement functions here

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5694533175892013116
xla_global_id: -1
]


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import composition_stats as cp

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, StratifiedKFold, train_test_split, LeaveOneOut
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, get_scorer, r2_score, mean_squared_error

from scipy.spatial.distance import pdist

import warnings
warnings.filterwarnings("ignore")

## COMBO dataset
Dataset downloaded from https://c-lasso.readthedocs.io/en/latest/auto_examples/plot_example_COMBO.html  

In [3]:
count = pd.read_csv('./datasets/COMBO_data/complete_data/GeneraCounts.csv', header=None)
count = count.transpose()

tax_table = pd.read_csv('./datasets/COMBO_data/complete_data/GeneraPhylo.csv', header=None)
tax_table = tax_table.transpose()
tax_table.columns = tax_table.iloc[0]
tax_table.drop(tax_table.index[0], inplace=True)
tax_table

,OTU_1,OTU_2,OTU_3,OTU_4,OTU_5,OTU_6,OTU_7,OTU_8,OTU_9,OTU_10,...,OTU_78,OTU_79,OTU_80,OTU_81,OTU_82,OTU_83,OTU_84,OTU_85,OTU_86,OTU_87
1,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,...,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria,Bacteria
2,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Bacteroidetes,Bacteroidetes,Bacteroidetes,...,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Proteobacteria,Synergistetes,Synergistetes,Verrucomicrobia
3,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Actinobacteria,Bacteroidia,Bacteroidia,Bacteroidia,...,Betaproteobacteria,Betaproteobacteria,Deltaproteobacteria,Epsilonproteobacteria,Gammaproteobacteria,Gammaproteobacteria,Gammaproteobacteria,Synergistia,Synergistia,Verrucomicrobiae
4,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Coriobacteriales,Bacteroidales,Bacteroidales,Bacteroidales,...,Burkholderiales,Neisseriales,Desulfovibrionales,Campylobacterales,Aeromonadales,Pseudomonadales,Xanthomonadales,Synergistales,Synergistales,Verrucomicrobiales
5,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Coriobacteriaceae,Bacteroidaceae,Porphyromonadaceae,Porphyromonadaceae,...,Oxalobacteraceae,Neisseriaceae,Desulfovibrionaceae,Campylobacteraceae,Succinivibrionaceae,Pseudomonadaceae,Xanthomonadaceae,Synergistaceae,Synergistaceae,Verrucomicrobiaceae
6,Asaccharobacter,Atopobium,Collinsella,Eggerthella,Gordonibacter,Olsenella,Slackia,Bacteroides,Barnesiella,Butyricimonas,...,Oxalobacter,Neisseria,Desulfovibrio,Campylobacter,Succinivibrio,Pseudomonas,Stenotrophomonas,Cloacibacillus,Pyramidobacter,Akkermansia


In [4]:
# BMI as response
Y = pd.read_csv('./datasets/COMBO_data/BMI.csv', header=None)
Y = np.array(Y).ravel()

# Zero ratio, make composotional
X = np.array(count, dtype=float)
print("zero ratio:", np.sum(X == 0) / X.size)
X = X / np.sum(X, axis=1)[:, None]

zero ratio: 0.7209051724137931


To make comparison with GBM zero replacement, we delete columns with only one nonzero value

In [5]:
valid_idx = np.sum(X > 0, axis=0) > 1

X = np.array(count.iloc[:, valid_idx], dtype=float)
print(X.shape)
print("zero ratio:", np.sum(X == 0) / X.size)
X = X / np.sum(X, axis=1)[:, None]

(96, 77)
zero ratio: 0.6860119047619048


## CV-MSE computation in conjunction with Kernel Ridge Regression
Rewrite this section using the Section "Using Column-deleted Data"

In [ ]:
for m in [3, 5, 10, 77]:
    cv_score(X, Y - Y.mean(0), "real-valued", num_features=m, model_name="krr", savepath="bmi_final",
             epsilon = 0.1, learning_rate = 0.001, 
             iterations = 5000, verbose=True,
             outer_folds = 5, inner_folds = 5, N_TRIALS=10,
             do_alpha=True, no_gamma=False, radial=True, oneSE=True)

In [14]:
selections = dict()
for m in [3, 5, 10, 77]:
    with open("./results/real_data/bmi_final/krr_neg_mean_squared_error_{}_1seTrue_radial.pickle".format(m), "rb") as fi:
        score, selection = pickle.load(fi)
    score_cv = np.mean(score, axis=1)
    print(np.mean(score_cv), np.std(score_cv)/np.sqrt(10))
    if m == 77:
        break
    selections[m] = selection_wrapper(selection, 77)
    # m = 77 (dim of X) is for comparison

28.90191912108356 0.053039353626440376
Selected features are: [15 40 48]
28.873559945159844 0.037140273387692165
Selected features are: [14 15 40 48 58]
28.98394520611011 0.11514238873274053
Selected features are: [ 9 14 15 40 48 54 56 58 63 69]
29.29507497131206 0.29381417600100346


In [15]:
for m in [3, 5, 10]:
    print("{} variables\n\n".format(m), tax_table.iloc[:, selections[m]['Selected_features']], "\n")

3 variables

 0          OTU_16            OTU_41            OTU_49
1        Bacteria          Bacteria          Bacteria
2   Bacteroidetes        Firmicutes        Firmicutes
3     Bacteroidia        Clostridia        Clostridia
4   Bacteroidales     Clostridiales     Clostridiales
5   Rikenellaceae   Lachnospiraceae   Ruminococcaceae
6       Alistipes             Dorea     Anaerotruncus 

5 variables

 0           OTU_15          OTU_16            OTU_41            OTU_49  \
1         Bacteria        Bacteria          Bacteria          Bacteria   
2    Bacteroidetes   Bacteroidetes        Firmicutes        Firmicutes   
3      Bacteroidia     Bacteroidia        Clostridia        Clostridia   
4    Bacteroidales   Bacteroidales     Clostridiales     Clostridiales   
5   Prevotellaceae   Rikenellaceae   Lachnospiraceae   Ruminococcaceae   
6       Prevotella       Alistipes             Dorea     Anaerotruncus   

0            OTU_59  
1          Bacteria  
2        Firmicutes  
3      

In [ ]:
new_score = list()
for m in [3, 5, 10, 77]:
    with open("./results/real_data/bmi_rm/krr_neg_mean_squared_error_{}_1seTrue_radial.pickle".format(m), "rb") as fi:
        score, selection = pickle.load(fi)

    score = cv_with_selection(X, Y - Y.mean(0), 'real-valued', selection, savepath="bmi3",
                              proj_method = 'compo', model_name='krr',
                              inner_folds=5, do_alpha=True, no_gamma=False, 
                              radial=True, oneSE=True)
    new_score.append(score)
new_score

In [ ]:
for m in [3, 5, 10, 77]:
    with open("./results/real_data/bmi3/krr_neg_mean_squared_error_{}_1seTrue_radial.pickle".format(m), "rb") as fi:
        score, selection = pickle.load(fi)
    score_cv = np.mean(score, axis=1)
    print(np.mean(score_cv), np.std(score_cv)/np.sqrt(10))

28.90191912108356 0.053039353626440376 (10,)
28.873559945159844 0.037140273387692165 (10,)
28.98394520611011 0.11514238873274053 (10,)
29.29507497131206 0.29381417600100346 (10,)


## Coda-lasso experiments with the c-lasso module

In [17]:
from classo import classo_problem
  
def learn(Z, Y, C=None, label=None, lam="theoretical"):
    # Z: log-transformed compositional data
    # Default C: zero sum constraint
    problem = classo_problem(Z, Y, C, label=label)
    problem.formulation.concomitant = False
    problem.formulation.huber = False
    problem.formulation.intercept = True

    problem.model_selection.CV = False
    problem.model_selection.ALO = False
    problem.model_selection.StabSel = False

    problem.model_selection.LAMfixed = True
    problem.model_selection.LAMfixedparameters.lam = lam
    problem.solve()
    
    return problem


def predict(model, X_new):
    beta = model.solution.LAMfixed.beta
    X_new = np.c_[np.ones(len(X_new)), X_new]
    return X_new @ beta

In [18]:
def cv_lasso_regression(X, Y, savepath='bmi_final', C=None, label=None, lamseq=np.geomspace(1e-3, 1, 100),
                   outer_folds = 5, inner_folds = 5, oneSE=False, N_TRIALS = 10, **krgs,
                   ):
    '''
    Input X should be zero-replaced count or compositional data
    label=None is almost always better.. it returns indices
    '''
    assert (X > 1e-10).all(), "Replace zeros or check the data type"
    X = np.log(X)
    
    if label is None:
        label = np.arange(X.shape[1])

    score_all = list()
    selection_all = list()
    
    for seed_ in range(1, N_TRIALS + 1):
        print("\n", seed_, "th cv experiment...")

        # Outer cv folds
        Outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=seed_)
        score = list()
        selection = list()
        
        # Outer cv loop
        for i, (train_idx, test_idx) in enumerate(Outer_cv.split(X)):
            print(f"\n Fold {i}:")
            X_train, Y_train = X[train_idx], Y[train_idx]
            X_test, Y_test = X[test_idx], Y[test_idx]

            if inner_folds is None:
                raise Exception("Not yet developed")
            else:
                # Do inner cv on X_train to choose lambda parameter
                inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=seed_ * N_TRIALS + i)
                cv_scores = np.zeros((len(lamseq), inner_folds))
                
                for j, (tr, val_idx) in enumerate(inner_cv.split(X_train)):
                    for k in range(len(lamseq)):
                        lam = lamseq[k]
                        model = learn(X_train[tr], Y_train[tr], lam=lam)
                        cv_scores[k, j] = mean_squared_error(Y_train[val_idx],
                                                            predict(model, X_train[val_idx]))
                
                # Parameter selection based on inner_cv result
                cv_scores_collected = cv_scores.mean(axis=1)
                
                if oneSE:
                    # Take maximal lambda among the [min, min+1se] scores
                    min_plus_1se = np.min(cv_scores_collected) + np.std(cv_scores_collected) / np.sqrt(len(cv_scores_collected))
                    mask = cv_scores_collected < min_plus_1se
                    lamb = lamseq[mask][-1]
                    # print("models:", lamseq[mask])
                else:
                    # find lambda "minimizing" cv-mse
                    lamb = lamseq[np.argmin(cv_scores_collected)]
                # Refit
                refitted_model = learn(X_train, Y_train, label=label, lam=lamb)
                
            # Get score
            refitted_score = mean_squared_error(Y_test,
                                                predict(refitted_model, X_test))
            selected_feats = label[refitted_model.solution.LAMfixed.selected_param[1:]]
            print("CV lambda, refitted score:", lamb, ',', refitted_score)
            print("Selected feats:", selected_feats)
            score.append(refitted_score)
            selection.append(selected_feats)
        
        # Wrap up the result of _ th iteration
        score_all.append(score)
        selection_all.append(selection)
    
    print(np.mean(score_all), "\n")
    
    # Save
    with open("./results/real_data/{}/cvlasso_inner{}_1se{}.pickle".format(savepath, inner_folds, oneSE), 'wb') as f:
        pickle.dump((score_all, selection_all), f)
    return score_all
        

In [ ]:
# 0.5min replacement
X_replaced = zero_replacement(X, method='min', val=1/2)
cv_lasso_regression(X_replaced, Y, savepath='bmi_final', label=None, inner_folds=5)

In [3]:
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_0.5min.pickle".format("bmi_final", 5, False), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() / np.sqrt(10)

(29.28863623980923, 0.2969653109467116)

In [4]:
selection

[[array([15, 52], dtype=int64),
  array([24, 52, 53], dtype=int64),
  array([15, 52], dtype=int64),
  array([24, 52, 53], dtype=int64),
  array([24, 48, 52, 62], dtype=int64)],
 [array([], dtype=int64),
  array([15, 52], dtype=int64),
  array([24, 52], dtype=int64),
  array([15, 24, 37, 52, 53], dtype=int64),
  array([48, 52, 53, 62], dtype=int64)],
 [array([24, 37, 48, 52, 55, 62], dtype=int64),
  array([52], dtype=int64),
  array([15, 52, 55], dtype=int64),
  array([15, 52], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([52], dtype=int64),
  array([15, 24, 48, 52, 55, 62, 66], dtype=int64),
  array([15, 24, 52, 55, 62], dtype=int64),
  array([], dtype=int64)],
 [array([52], dtype=int64),
  array([15, 24, 49, 52, 53, 55, 61], dtype=int64),
  array([], dtype=int64),
  array([24, 52], dtype=int64),
  array([52], dtype=int64)],
 [array([], dtype=int64),
  array([24, 52, 53, 55, 62], dtype=int64),
  array([15, 24, 49, 52, 53, 55], dtype=int64),
  array([52], d

As shown below, applying 1se rule for CV returns almost no selection but intercept (lasso isn't prone to overfitting)  
We only record CV result without applying 1se rule for lasso

In [ ]:
# 1se applied
X_replaced = zero_replacement(X, method='min', val=1/2)
cv_lasso_regression(X_replaced, Y, savepath='bmi_final', label=None, inner_folds=5, oneSE=True)

In [25]:
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_0.5min.pickle".format("bmi_final", 5, True), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() / np.sqrt(10)

(29.58014152071525, 0.16147063989710317)

In [26]:
# lasso with 1se rule asserts almost no selection is the best
selection

[[array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([24, 48, 52]),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([24, 56]),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dty

### 1sum replacement

In [ ]:
# 1sum replacement
X_replaced = zero_replacement(X, method='sum', val=1.)
cv_lasso_regression(X_replaced, Y, savepath='bmi_final', label=None, inner_folds=5)

In [5]:
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_1sum.pickle".format("bmi_final", 5, False), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() / np.sqrt(10)

(30.516218621834316, 0.37851058876287474)

In [6]:
selection

[[array([15, 56], dtype=int64),
  array([], dtype=int64),
  array([14, 15, 48, 56], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([14, 15, 56], dtype=int64),
  array([ 8, 10, 36, 40, 46, 48, 49, 54, 56, 58, 62, 69], dtype=int64),
  array([14, 15, 56], dtype=int64),
  array([ 7, 14, 15, 40, 48, 56], dtype=int64)],
 [array([ 7,  8, 14, 15, 40, 46, 48, 49, 54, 56], dtype=int64),
  array([26, 36, 40, 46, 48, 49, 54, 56, 58, 62], dtype=int64),
  array([], dtype=int64),
  array([ 7, 14, 15, 40, 48, 56], dtype=int64),
  array([ 7, 11, 15, 46], dtype=int64)],
 [array([], dtype=int64),
  array([ 8, 10, 15, 26, 36, 40, 46, 48, 49, 54, 56, 58, 69, 70],
        dtype=int64),
  array([ 7, 14, 15, 40, 46, 48, 56, 58], dtype=int64),
  array([ 7, 14, 15, 56], dtype=int64),
  array([14, 15, 40, 48, 49, 56], dtype=int64)],
 [array([], dtype=int64),
  array([ 7, 14, 15, 36, 40, 48, 49, 54, 56, 58], dtype=int64),
  array([], dtype=int64),
  array([ 7,

### GBM replacement

In [ ]:
# Used R package zCompositions
x_gbm = pd.read_csv('./datasets/COMBO_data/complete_data/GeneraCountsGBM.csv')
x_gbm = np.array(x_gbm.iloc[:, 1:])
cv_lasso_regression(x_gbm, Y, savepath='bmi_final', label=None, inner_folds=5)

In [9]:
# gbm result
with open("./results/real_data/{}/cvlasso_inner{}_1se{}_gbm.pickle".format("bmi_final", 5, False), 'rb') as fi:
    score, selection = pickle.load(fi)
mse_through_repeat = np.mean(score, axis=1)
mse_through_repeat.mean(), mse_through_repeat.std() / np.sqrt(10)

(29.052951227592416, 0.44015695139933747)

In [8]:
selection

[[array([15, 52], dtype=int64),
  array([24, 48, 52, 53], dtype=int64),
  array([15, 53], dtype=int64),
  array([52, 53], dtype=int64),
  array([24, 52, 53], dtype=int64)],
 [array([], dtype=int64),
  array([52], dtype=int64),
  array([24, 52], dtype=int64),
  array([ 0, 15, 24, 31, 37, 52, 53], dtype=int64),
  array([48, 53, 61], dtype=int64)],
 [array([24, 37, 48, 52, 53], dtype=int64),
  array([53, 56], dtype=int64),
  array([ 0, 15, 52, 53], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)],
 [array([], dtype=int64),
  array([15, 24, 52, 53], dtype=int64),
  array([ 0, 24, 48, 52, 53], dtype=int64),
  array([ 0, 15, 24, 52], dtype=int64),
  array([], dtype=int64)],
 [array([52], dtype=int64),
  array([15, 24, 31, 52, 53, 55, 61], dtype=int64),
  array([], dtype=int64),
  array([53], dtype=int64),
  array([52], dtype=int64)],
 [array([], dtype=int64),
  array([24, 48, 53, 56], dtype=int64),
  array([ 0, 53], dtype=int64),
  array([48, 52, 53], dtype=int64),
  array([